# Visier Platform SDK Demo (Notebook)

This notebook shows a minimal, reproducible flow for configuring and calling the Visier Platform Python SDK. Fill in your tenant-specific environment variables before running requests.

**Prerequisites**
- Python 3.9+
- `visier-platform-sdk`
- `python-dotenv` (optional, for loading a local `.env`)

**Environment variables** (see `visier.env.example`):
- `VISIER_HOST`, `VISIER_APIKEY`, `VISIER_VANITY`
- Add auth fields your tenant requires: `VISIER_CLIENT_ID`, `VISIER_CLIENT_SECRET`, `VISIER_USERNAME`, `VISIER_PASSWORD`, `VISIER_TENANT_CODE` (optional)

> Never hard-code secrets in the notebook. Use environment variables or a secrets manager.



In [1]:
# Step 0: Install Dependencies (uncomment if needed)
# Run this cell first if you haven't installed the SDK yet

# %pip install --upgrade pip
# %pip install visier-platform-sdk python-dotenv



In [ ]:
# Step 1: Load Environment Variables
# This cell loads your credentials from a .env file (if available)

from pathlib import Path

try:
    from dotenv import load_dotenv
    env_path = Path(".env")
    if env_path.exists():
        load_dotenv(env_path)
        print(f"✓ Loaded environment variables from {env_path}")
    else:
        print("ℹ No .env file found; using system environment variables.")
        print("  Make sure your credentials are set in your environment.")
except ImportError:
    print("ℹ python-dotenv not installed; ensure env vars are set externally.")
    print("  Install with: pip install python-dotenv")



In [ ]:
# Step 2: Create API Client
# This cell creates and configures the Visier API client using your credentials

from visier_platform_sdk import ApiClient, Configuration
from visier_platform_sdk.exceptions import ApiException, ApiValueError

try:
    # Create configuration from environment variables
    # The SDK automatically reads:
    # - VISIER_HOST, VISIER_APIKEY, VISIER_VANITY (required)
    # - VISIER_USERNAME, VISIER_PASSWORD (required for Basic Auth)
    config = Configuration.from_env()
    
    # Set this configuration as the default for all API instances
    Configuration.set_default(config)
    
    # Create the API client
    client = ApiClient(config)
    
    print("✓ Configuration created successfully")
    print(f"  Host: {config.host}")
    print(f"  API Key: {'*' * 20} (hidden)")
    if hasattr(config, 'vanity') and config.vanity:
        print(f"  Vanity: {config.vanity}")
    if hasattr(config, 'username') and config.username:
        print(f"  Username: {config.username}")
        print(f"  Password: {'*' * 20} (hidden)")
        print(f"  Authentication: Username/Password (Basic Auth)")
    
except ApiValueError as e:
    print(f"✗ Configuration error: {e}")
    print("\nPlease ensure your .env file contains all required variables:")
    print("  - VISIER_HOST")
    print("  - VISIER_APIKEY")
    print("  - VISIER_VANITY")
    print("  - VISIER_USERNAME")
    print("  - VISIER_PASSWORD")
except Exception as e:
    print(f"✗ Unexpected error: {e}")


In [3]:
# Step 3: Explore the Data Model
# This cell demonstrates how to list analytic objects and view their properties

from typing import Iterable, Any
from visier_platform_sdk import DataModelApi

# Initialize the DataModelApi
data_model_api = DataModelApi(client)

# Example 1: List all analytic objects
print("=" * 70)
print("Example 1: Listing all analytic objects")
print("=" * 70)
try:
    analytic_objects = data_model_api.analytic_objects()
    
    if hasattr(analytic_objects, 'objects') and analytic_objects.objects:
        print(f"✓ Found {len(analytic_objects.objects)} analytic objects\n")
        print("Sample analytic objects:")
        for obj in analytic_objects.objects[:5]:  # Show first 5
            obj_id = getattr(obj, 'id', 'N/A')
            obj_type = getattr(obj, 'object_type', 'N/A')
            print(f"  - {obj_id} (type: {obj_type})")
        if len(analytic_objects.objects) > 5:
            print(f"  ... and {len(analytic_objects.objects) - 5} more")
    else:
        print("ℹ No analytic objects found or response structure differs")
except ApiException as e:
    print(f"✗ API error: {e.status} - {e.reason}")
    if e.body:
        print(f"  Details: {e.body}")

print("\n")

# Example 2: Get properties for a specific analytic object
print("=" * 70)
print("Example 2: Getting properties for 'Employee' analytic object")
print("=" * 70)

analytic_object_id = "Employee"  # Change this to match your tenant
max_properties_to_show = 5

try:
    print(f"Requesting properties for analytic object '{analytic_object_id}'...")
    properties = data_model_api.properties(analytic_object_id)
    
    total = len(properties) if hasattr(properties, "__len__") else "unknown"
    print(f"✓ Found {total} properties\n")
    
    # Helper function to extract property names
    def get_property_names(items: Iterable[Any], limit: int):
        names = []
        for idx, item in enumerate(items):
            if idx >= limit:
                break
            name = getattr(item, "name", None) or getattr(item, "id", None) or repr(item)
            prop_type = getattr(item, "data_type", None) or getattr(item, "type", None) or ""
            names.append((name, prop_type))
        return names
    
    sample = get_property_names(properties, max_properties_to_show)
    if sample:
        print("Sample properties:")
        for name, prop_type in sample:
            type_str = f" ({prop_type})" if prop_type else ""
            print(f"  - {name}{type_str}")
        if total != "unknown" and total > max_properties_to_show:
            print(f"  ... and {total - max_properties_to_show} more properties")
    else:
        print("No properties returned.")
        
except ApiException as e:
    print(f"✗ API error: {e.status} - {e.reason}")
    if e.body:
        print(f"  Details: {e.body}")
    print(f"\n  Note: '{analytic_object_id}' may not exist in your tenant.")
    print("  Try listing all objects first (Example 1) to see what's available.")



/Users/lakshman.rajanathan/Desktop/Cursor/PythonSDK/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'client' is not defined

In [ ]:
# Step 4: Query Properties Using List Query
# This cell demonstrates how to query specific properties from Employee data

from visier_platform_sdk import DataQueryApi
from visier_platform_sdk.models import (
    ListQueryExecutionDTO,
    ListQuerySourceDTO,
    PropertyColumnDTO,
    QueryPropertyDTO,
    PropertyReferenceDTO,
    SelectionConceptReferenceDTO
)
from visier_platform_sdk.exceptions import ApiException

# Initialize the DataQueryApi
query_api = DataQueryApi(client)

# Define the columns to query
# Example: Time in Role, Direct SOC, and Manager flag
columns = [
    PropertyColumnDTO(
        column_name="Time in Role",
        column_definition=QueryPropertyDTO(
            var_property=PropertyReferenceDTO(
                name="Employee.Time_in_Role",
                qualifying_path="Employee"
            )
        )
    ),
    PropertyColumnDTO(
        column_name="Direct SOC",
        column_definition=QueryPropertyDTO(
            var_property=PropertyReferenceDTO(
                name="Employee.Span_Of_Control",
                qualifying_path="Employee"
            )
        )
    ),
    PropertyColumnDTO(
        column_name="Manager flag",
        column_definition=QueryPropertyDTO(
            selection_concept=SelectionConceptReferenceDTO(
                id="isManager",
                qualifying_path="Employee"
            )
        )
    )
]

# Create the list query
list_query = ListQueryExecutionDTO(
    source=ListQuerySourceDTO(
        analytic_object="Employee"
    ),
    columns=columns
)

print("=" * 70)
print("Querying Employee Properties")
print("=" * 70)
print(f"\nQuerying {len(columns)} columns:")
for col in columns:
    print(f"  - {col.column_name}")
print()

try:
    # Execute the list query
    result = query_api.list_query(list_query)
    
    # Check if result has data
    if hasattr(result, 'data') and result.data:
        print(f"✓ Query successful! Found {len(result.data)} rows\n")
        
        # Display first few rows
        print("Sample data (first 5 rows):")
        for idx, row in enumerate(result.data[:5]):
            print(f"  Row {idx + 1}: {row}")
        if len(result.data) > 5:
            print(f"  ... and {len(result.data) - 5} more rows")
    elif hasattr(result, 'rows') and result.rows:
        print(f"✓ Query successful! Found {len(result.rows)} rows\n")
        print("Sample data (first 5 rows):")
        for idx, row in enumerate(result.rows[:5]):
            print(f"  Row {idx + 1}: {row}")
        if len(result.rows) > 5:
            print(f"  ... and {len(result.rows) - 5} more rows")
    else:
        print("ℹ Query executed but no data returned")
        print(f"Result structure: {type(result)}")
        if hasattr(result, '__dict__'):
            print(f"Result attributes: {list(result.__dict__.keys())}")
        
except ApiException as e:
    print(f"✗ API error: {e.status} - {e.reason}")
    if e.body:
        print(f"  Details: {e.body}")
    print("\nNote: Make sure the property names and selection concepts exist in your tenant.")
    print("      You may need to adjust the property names to match your data model.")



### Tips
- For CSV responses, set a default header: `client.set_default_header('Accept', 'text/csv')` or pass `_headers={'Accept': 'text/csv'}` per call.
- Swap `analytic_object_id` to match your tenant (e.g., `Applicant`, `Customer`).
- This SDK uses Basic Auth (username/password). See the SDK README for details: https://github.com/visier/python-sdk.



## Next Steps

1. **Explore More APIs**: Try other APIs like `AnalyticObjectsV2Api`, `MetricsV2Api`, `ConceptsV2Api`
2. **Build Queries**: Learn to construct aggregation and list queries using `DataQueryApi`
3. **Read the Documentation**: Check the SDK source code in `venv/lib/python3.*/site-packages/visier_platform_sdk/`
4. **Check API Models**: Explore available DTOs in `visier_platform_sdk/models/`
5. **Run the Full Walkthrough**: Execute `visier_sdk_walkthrough.py` for a comprehensive guide

## Additional Resources

- **SDK Documentation**: Browse the API classes in `visier_platform_sdk/api/`
- **Official Visier API Docs**: Refer to the [Visier API documentation](https://docs.visier.com/developer/)
- **SDK Repository**: Check the [official Python SDK repository](https://github.com/visier/python-sdk)
- **More Examples**: See [github.com/visier/api-samples](https://github.com/visier/api-samples) for additional examples
- **Support**: Contact your Customer Success Manager (CSM) for API access questions

---

**Happy coding! 🚀**
